In [1]:
import pandas as pd

In [2]:
import os

In [3]:
import sys

In [4]:
import itertools

In [5]:
%load_ext autoreload
%autoreload 2

------------------------------

## Config

In [6]:
#project_data_dir = '/data/pablo/RNAdeg'
project_data_dir = '/gcm-lfs1/pablo/data/RNAdeg'

In [7]:
project_dir = '/home/pmonteagudo/workspace/RNAdeg'

In [8]:
scripts_dir = os.path.join(project_dir, 'pyRNAdeg')
if scripts_dir not in sys.path: 
    sys.path.append(scripts_dir)

import RepTools as rpc    
from Util import to_log2_tpm

In [21]:
#source_dir = '/data/parastou/RNAdeg/results/RipChip/xp_data/'
#source_dir = os.path.join(project_data_dir,'results/RipChip/xp_data')
#in_dir = os.path.join(project_dir, 'results/ChIP/xp_data')
in_dir = os.path.join(project_dir, 'results/sequencing_new/ChIP/xp_data')

In [24]:
#out_dir = '/data/parastou/RNAdeg/results/RipChip/'
#out_dir = os.path.join(project_data_dir,'results/RipChip')
#out_dir = os.path.join(project_dir, 'results/ChIP')
out_dir = os.path.join(project_dir, 'results/sequencing_new/ChIP')

------

# Substract INPUT: **ChIP-seq**

--------------------------

For now just select the **pair (ChIP-INPUT)** that were processed with Thor. 
- `sequencing_new/ChIP/bam/524_1_S2ChIP`
- `sequencing_new/ChIP/bam/524_1_INPUT`

In [16]:
chip_sample = "524_1_S2ChIP"
input_sample = "524_1_INPUT"

## 1. Load raw gene count table

In [25]:
gx_header = ['gene_id', 'gene_name', 'seqid', 'type', 'start', 'end', 'strand', 'cds_length', 'utr_length', 'length']
gx_header.append(chip_sample)
gx_header.append(input_sample)

- Import **ChIP** (raw) **gene counts Matrix**: `chip_pombe_gene_count_matrix.csv`

In [26]:
gx_df = pd.read_csv(os.path.join(in_dir, 'chip_pombe_gene_count_matrix.csv'), sep='\t', comment='#')
#gx_df = pd.read_csv(os.path.join(in_dir, 'chip_pombe_gene_count_matrix.csv'), sep='\t', comment='#')[gx_header]
gx_df.head()

,gene_id,gene_name,seqid,type,start,end,strand,cds_length,utr_length,length,...,491_1_INPUT,491_OIN,504_1_INPUT,510_1_INPUT,523_1_INPUT,524_1_INPUT,544_1_OIN,544_2_OIN,638_OIN,63_OIN
0,SPAC1002.01,mrx11,I,gene,1798347,1799015,+,540,NaN,540.0,...,506.0,365.0,470.0,577.0,440.0,394.0,277.0,425.0,270.0,245.0
1,SPAC1002.02,pom34,I,gene,1799061,1800053,+,690,303.0,993.0,...,578.0,455.0,584.0,629.0,539.0,501.0,385.0,503.0,337.0,361.0
2,SPAC1002.03c,gls2,I,gene,1799915,1803141,-,2772,455.0,3227.0,...,1669.0,1353.0,1736.0,2057.0,1522.0,1460.0,1206.0,1410.0,1014.0,1183.0
3,SPAC1002.04c,taf11,I,gene,1803624,1804491,-,600,268.0,868.0,...,575.0,481.0,529.0,611.0,489.0,472.0,348.0,449.0,316.0,345.0
4,SPAC1002.05c,jmj2,I,gene,1804548,1806797,-,2148,102.0,2250.0,...,1226.0,1052.0,1237.0,1501.0,1181.0,1135.0,850.0,997.0,728.0,785.0


In [16]:
gx_df.shape

(7005, 34)

- Get `samples` **columns**

In [31]:
sample_cols = [ss for ss in gx_df.columns if ('ChIP' in ss)| ('INPUT' in ss) | ('OIN' in ss)]
sample_cols
#len(sample_cols)

['1022_1_INPUT',
 '1022_OIN',
 '1168_OIN',
 '260_1_INPUT',
 '301_OIN',
 '302_OIN',
 '324_1_INPUT',
 '324_OIN',
 '491_1_INPUT',
 '491_OIN',
 '504_1_INPUT',
 '510_1_INPUT',
 '523_1_INPUT',
 '524_1_INPUT',
 '544_1_OIN',
 '544_2_OIN',
 '638_OIN',
 '63_OIN']

----

## 2. Produce **correlation-plots** and **pearson-r correlation scores**

- Get `samples` **columns**

In [27]:
sample_cols = [ss for ss in log2_tpm_df.columns if 'ChIP' in ss]
#sample_cols
len(sample_cols)

NameError: name 'log2_tpm_df' is not defined

- **Replicate samples** grouped by `Mutant` type:

In [43]:
sub_samples =  {'638_ChIP': ['638_S2_ChIP', '638ChIP_1'],
                '1168_ChIP': ['1168_S2ChIP', '1168_S2ChIP_1'],
                '63_ChIP': ['63_S2ChIPp'], 
                '302_ChIP': ['302_S2ChIP', '302_S2_ChIP'],
                '301_ChIP': ['301_S2ChIP', '301_S2_ChIP'],
                '324_ChIP': ['324_S2ChIP', '324_S2_ChIP'],
                '80_ChIP': ['80_S2_ChIP', '80S2ChIP_1', '80_S2ChIP', '80_S2ChIP_2'],
                '504_ChIP': ['504S2ChIP_1', '504S2ChIP_2'],
                '591_ChIP': ['591_S2PChIP'],
                '491_ChIP': ['491_S2ChIP', '491_S2_ChIP'],
                '530_ChIP': ['530ChIP_1', '530S2ChIP_2']}

In [68]:
sample_cols_v2 = list(itertools.chain.from_iterable(sub_samples.values()))
#len(sample_cols_v2)
missing_samples = set(sample_cols_v2).difference(sample_cols)
missing_samples

{'1168_pA', '1168_pA_2', '63_S2Ph_RIP'}

- Run **correlation checks**  for all **pair-wise sample combinations**:

In [40]:
corr_df = rpc.run_corr_checks(log2_tpm_df, out_dir=out_corr)
corr_df.head()

> /home/pmonteagudo/workspace/RNAdeg/pyRNAdeg/RepTools.py(113)run_corr_checks()
-> if not samples:


(Pdb)  samples
(Pdb)  q


BdbQuit: 

In [19]:
corr_df.shape

(231, 4)

-----------------

## 3. Investigate **pearson-r correlation scores** and select replicates to merge

Select replicates to be merged based on their **pearson-r correlation scores**

In [20]:
from RepTools import report_corr

- **Replicate samples** grouped by `Mutant` type:

In [22]:
# sub_samples =  [['638_S2_ChIP', '638ChIP_1'] ,
#                 ['63_S2ChIPp'],
#                 ['302_S2ChIP', '302_S2_ChIP'],
#                 ['301_S2_ChIP', '301_S2ChIP'],
#                 ['324_S2ChIP', '324_S2_ChIP'], 
#                 ['80_S2_ChIP', '80_S2Ph_ChIP', '80S2ChIP_1', '80_S2ChIP_2', '80_S2ChIP'],
#                 ['504S2ChIP_1', '504S2ChIP_2'],
#                 ['591_S2PChIP'],
#                 ['491_S2ChIP', '491_S2_ChIP'],
#                 ['530ChIP_1', '530S2ChIP_2']]

**Total number of samples**:

In [23]:
len(list(itertools.chain.from_iterable(sub_samples)))

21

**Number of mutants**:

In [29]:
len(sub_samples)

11

- Import **correlation checks** for all **pair-wise sample combinations**: `correlations.csv`

In [35]:
corr_df = pd.read_csv(os.path.join(out_corr, 'correlations.csv'), sep='\t')
corr_df.head()

,Sample1,Sample2,Correlation,P-value
0,1168_S2ChIP_1,1168_S2ChIP,NaN,1.0
1,1168_S2ChIP_1,301_S2_ChIP,NaN,1.0
2,1168_S2ChIP_1,301_S2ChIP,NaN,1.0
3,1168_S2ChIP_1,302_S2_ChIP,NaN,1.0
4,1168_S2ChIP_1,302_S2ChIP,NaN,1.0


In [36]:
corr_df.shape

(231, 4)

- Check **correlation scores** grouped by `Mutant` type:

In [38]:
for s in sub_samples:
    a = report_corr(corr_df, s)
    
    print('-'*80)
    print('Mutant samples group:', s)
    print(a)
    print('-'*80, '\n')

--------------------------------------------------------------------------------
Mutant samples group: ['638_S2_ChIP', '638ChIP_1']
     Sample1      Sample2  Correlation  P-value
0  638ChIP_1  638_S2_ChIP          NaN      1.0
-------------------------------------------------------------------------------- 

--------------------------------------------------------------------------------
Mutant samples group: ['1168_S2ChIP', '1168_S2ChIP_1']
         Sample1      Sample2  Correlation  P-value
0  1168_S2ChIP_1  1168_S2ChIP          NaN      1.0
-------------------------------------------------------------------------------- 

--------------------------------------------------------------------------------
Mutant samples group: ['63_S2ChIPp']
Empty DataFrame
Columns: [Sample1, Sample2, Correlation, P-value]
Index: []
-------------------------------------------------------------------------------- 

--------------------------------------------------------------------------------
Mutant s

-----------------

## 4. Merge replicates

Merge and store as `.csv` files:
- **Average raw count tables** (gx_df) for replicates: `chip_pombe_gene_count_merged.csv`
- **Average TPM-normalized expression tables** (tpm_df) for replicates: `chip_pombe_gene_count_merged.csv`

- `Mutant` type columns:

In [26]:
col_names = ['638_ChIP', '1160_ChIP', 'WT_ChIP', '302_ChIP', '301_ChIP', '324_ChIP', 
             '80_ChIP', '504_ChIP', '591_ChIP', '491_ChIP', '530_ChIP']

- **Replicate samples** grouped by `Mutant` type:

In [27]:
sub_samples =  [['638_S2_ChIP', '638ChIP_1'] ,
                ['1168_S2ChIP', '1168_S2ChIP_1'], 
                ['63_S2ChIPp'],
                ['302_S2ChIP', '302_S2_ChIP'],
                ['301_S2_ChIP', '301_S2ChIP'], 
                ['324_S2ChIP', '324_S2_ChIP'],
                ['80_S2_ChIP', '80S2ChIP_1', '80_S2ChIP_2', '80_S2ChIP'],
                ['504S2ChIP_1', '504S2ChIP_2'],
                ['591_S2PChIP'],
                ['491_S2ChIP', '491_S2_ChIP'], 
                ['530ChIP_1', '530S2ChIP_2']]

**Total number of samples**:

In [28]:
len(list(itertools.chain.from_iterable(sub_samples)))

22

**Number of mutants**:

In [29]:
len(sub_samples)

11

- **Average raw count tables (gx_df) for replicates**

In [30]:
#merged_df = rpc.repli_merge(gx_df, sub_samples, new_cols=col_names, out_dir=in_dir, out_file='chip_pombe_gene_count_merged.csv')
merged_gxp_df = rpc.repli_merge(gx_df, sub_samples, new_cols=col_names, out_dir=in_dir, out_file='chip_pombe_gene_count_merged.csv')
merged_gxp_df.head()

,gene_id,gene_name,length,type,category,bio_type,638_ChIP,1160_ChIP,WT_ChIP,302_ChIP,301_ChIP,324_ChIP,80_ChIP,504_ChIP,591_ChIP,491_ChIP,530_ChIP
0,SPAC1002.01,mrx11,540.0,gene,gene,mRNA,151.5,39.0,107.0,99.0,91.5,82.0,156.00,72.5,155.0,118.5,84.5
1,SPAC1002.02,pom34,993.0,gene,gene,mRNA,577.5,177.0,406.0,411.5,410.0,362.0,533.75,274.0,885.0,409.5,357.5
2,SPAC1002.03c,gls2,3227.0,gene,gene,mRNA,1409.0,478.0,1172.0,1197.0,1063.0,994.5,1367.75,827.0,2138.0,1253.0,919.5
3,SPAC1002.04c,taf11,868.0,gene,gene,mRNA,262.0,170.0,320.0,273.5,198.5,244.0,301.50,171.5,453.0,267.0,220.0
4,SPAC1002.05c,jmj2,2250.0,gene,gene,mRNA,673.0,79.0,673.0,451.5,492.0,430.0,620.25,404.0,764.0,586.0,468.0


In [31]:
merged_gxp_df.shape

(7005, 17)

- **B. Average TPM-normalized expression tables (tpm_df) for replicates**

In [32]:
merged_tpm_df = rpc.repli_merge(tpm_df, sub_samples, new_cols=col_names, out_dir=in_dir, out_file='chip_pombe_tpm_merged.csv')
merged_tpm_df.head()

,gene_id,gene_name,length,type,category,bio_type,638_ChIP,1160_ChIP,WT_ChIP,302_ChIP,301_ChIP,324_ChIP,80_ChIP,504_ChIP,591_ChIP,491_ChIP,530_ChIP
0,SPAC1002.01,mrx11,540.0,gene,gene,mRNA,70.364681,49.156389,67.171057,59.624368,67.644011,51.053360,73.288988,54.728487,53.046393,79.251067,60.371322
1,SPAC1002.02,pom34,993.0,gene,gene,mRNA,145.683010,121.320207,138.601826,131.428594,161.765092,127.065838,140.492491,112.243348,164.706956,136.077102,132.585836
2,SPAC1002.03c,gls2,3227.0,gene,gene,mRNA,110.207094,100.817995,123.117790,118.673575,128.339363,108.855655,108.619941,103.873637,122.440940,127.013853,102.302088
3,SPAC1002.04c,taf11,868.0,gene,gene,mRNA,75.733997,133.302508,124.974791,101.195329,90.280554,105.960388,89.637450,80.191357,96.448704,101.235949,91.751409
4,SPAC1002.05c,jmj2,2250.0,gene,gene,mRNA,76.146466,23.897567,101.396908,64.395437,85.134130,70.657833,71.490491,72.663338,62.752172,86.678594,75.273765


In [33]:
merged_tpm_df.shape

(7005, 17)

------